In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [5]:
# 1. 데이터 불러오기
df = pd.read_csv("../../data/DRM-FLOODSHELTER/DRM-FLOODSHELTER.csv")  # 공공데이터는 cp949 가능성 ↑
print("원본 크기:", df.shape)
display(df.head())

# 2. 구 컬럼 정리
# df["gu"] = df["SGG_NM"].astype(str).str.replace("구$", "", regex=True)
df.rename(columns={'SGG_NM':'gu'}, inplace=True)

# 3. 수치형 변환 (수용인원)
df["QTY_CPTY"] = pd.to_numeric(df["QTY_CPTY"], errors="coerce").fillna(0)

# 4. 집계 (구별)
agg = df.groupby("gu").agg(
    floodshelter_count=("EQUP_NM", "count"),   # 대피소 개수
    floodshelter_total_capacity=("QTY_CPTY", "sum"),  # 총 수용인원
    floodshelter_avg_capacity=("QTY_CPTY", "mean")    # 평균 수용인원
).reset_index()

display(agg.head())

# 5. 점수화 (많을수록 안전 → 정방향)
scaler = MinMaxScaler(feature_range=(0,100))
scaled = scaler.fit_transform(agg[["floodshelter_count","floodshelter_total_capacity","floodshelter_avg_capacity"]])

score_df = agg[["gu"]].copy()
for i, col in enumerate(["floodshelter_count","floodshelter_total_capacity","floodshelter_avg_capacity"]):
    score_df[col+"_score"] = scaled[:, i]

# 6. 종합 점수 (평균)
score_df["floodshelter_mean_score"] = score_df.filter(like="_score").mean(axis=1)

# 7. 저장
score_df.to_csv("../../result/DRM/DRM-FLOODSHELTER_result.csv", index=False, encoding="utf-8-sig")

score_df.head()

원본 크기: (1125, 19)


,R_SEQ_NO,CD_AREA,NO_EQUP_SE,SD_NM,SGG_NM,CD_EQUP,GB_ACMD,EQUP_NM,RDNMADR_CD,BDONG_CD,HDONG_CD,LOC_SFPR_A,SECT_EQUP,QTY_CPTY,XCORD,YCORD,XX,YY,CD_GUBUN
0,3751,1156000000,416,서울특별시,영등포구,1,학교,양화중학교,1156012100100480000037220,1.156012e+09,1156060500,서울특별시 영등포구 당산로 56(문래동3가),725,279,126.89605,37.51977,190810.6356,546703.3200,긴급
1,3800,1159000000,11,서울특별시,동작구,1,학교,서울신상도초등학교,1159010200102040333026309,1.159010e+09,1159054000,서울특별시 동작구 장승배기로 14(상도동),530,203,126.94406,37.50028,195053.8134,544536.2178,안전
2,3798,1156000000,461,서울특별시,영등포구,14,"공공시설(국·공립도서관, 공립병원, 시·도민회관, 구민회관 주민체육시설, 노인병원,...",푸드뱅크마켓3호점,1156013200147290000009413,1.156013e+09,1156068000,서울특별시 영등포구 신길로8길 7(신길동),62,24,126.90933,37.49573,191982.1121,544034.2814,안전
3,3763,1156000000,445,서울특별시,영등포구,3,경로당,구립문래제1경로당,1156012100100730001000001,1.156012e+09,1156060500,서울특별시 영등포구 문래로20길 7(문래동3가),201,77,126.89420,37.51846,190646.4768,546558.4688,안전
4,3799,1156000000,421,서울특별시,영등포구,1,학교,서울영문초등학교,1156012400100130000037490,1.156012e+09,1156060500,서울특별시 영등포구 문래로 56(문래동6가),720,277,126.88754,37.51960,190058.5474,546685.1817,긴급


,gu,floodshelter_count,floodshelter_total_capacity,floodshelter_avg_capacity
0,강남구,35,63780,1822.285714
1,강동구,50,49144,982.880000
2,강북구,29,19998,689.586207
3,강서구,88,29316,333.136364
4,관악구,41,12802,312.243902


,gu,floodshelter_count_score,floodshelter_total_capacity_score,floodshelter_avg_capacity_score,floodshelter_mean_score
0,강남구,13.114754,51.277703,64.746311,43.046256
1,강동구,37.704918,37.862734,31.762776,35.776809
2,강북구,3.278689,11.148283,20.238115,11.555029
3,강서구,100.000000,19.688915,6.231806,41.973574
4,관악구,22.950820,4.552620,5.410860,10.971433


gu (자치구명)

floodshelter_count_score (대피소 개수 점수)

floodshelter_total_capacity_score (총 수용인원 점수)

floodshelter_avg_capacity_score (평균 수용인원 점수)

floodshelter_mean_score (종합 평균 점수)